In [10]:
import keras
from keras.models import Sequential
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import numpy as np
from numpy import loadtxt
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report, precision_score, accuracy_score
from sklearn.model_selection import StratifiedKFold as kfold
from keras.layers import Dropout
from sklearn import preprocessing
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
dataset = pd.read_csv("/content/drive/MyDrive/Dataset/nn_data.txt", header=None)
dataset

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [8]:
min_max_scale = preprocessing.MinMaxScaler().fit(dataset)
min_max_scaled_dataset = min_max_scale.transform(dataset)
min_max_scaled_dataset

array([[0.35294118, 0.74371859, 0.59016393, ..., 0.23441503, 0.48333333,
        1.        ],
       [0.05882353, 0.42713568, 0.54098361, ..., 0.11656704, 0.16666667,
        0.        ],
       [0.47058824, 0.91959799, 0.52459016, ..., 0.25362938, 0.18333333,
        1.        ],
       ...,
       [0.29411765, 0.6080402 , 0.59016393, ..., 0.07130658, 0.15      ,
        0.        ],
       [0.05882353, 0.63316583, 0.49180328, ..., 0.11571307, 0.43333333,
        1.        ],
       [0.05882353, 0.46733668, 0.57377049, ..., 0.10119556, 0.03333333,
        0.        ]])

In [14]:
precisionScores = []
accuracyScores = []
X = dataset.iloc[:,0:8]
y = dataset.iloc[:,8]
folds = kfold(n_splits=10)
folds

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)

In [15]:
def ann_model(x_train,y_train,x_test,y_test,epochs,batch_size) :
    model = Sequential();
    model.add(Dense(8, input_shape=(8,), activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile( optimizer = 'adam' , loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.fit(x_train,y_train,epochs=epochs,batch_size = batch_size)
    yp = model.predict(x_test)
    y_pred = [] ;
    for element in yp:
        if element > 0.5 :
            y_pred.append(1)
        else :
            y_pred.append(0)
    print(classification_report(y_test,y_pred))
    precisionScores.append(precision_score(y_test, y_pred))
    accuracyScores.append(accuracy_score(y_test, y_pred))
    

In [16]:
def ann_model_without_k_fold(X,y,epochs,batch_size) :
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
    model = Sequential();
    model.add(Dense(8, input_shape=(8,), activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile( optimizer = 'adam' , loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.fit(X_train,y_train,epochs=epochs,batch_size = batch_size)
    yp = model.predict(X_test);
    y_pred = [] ;
    for element in yp:
        if element > 0.5 :
            y_pred.append(1)
        else :
            y_pred.append(0)
    precisionScore = precision_score(y_test, y_pred)
    accuracyScore = accuracy_score(y_test, y_pred)
    return precisionScore,accuracyScore ;

In [ ]:
precisionScore,accuracyScore = ann_model_without_k_fold(X,y,100,20)

In [18]:
print("Precision", precisionScore)
print("Accuracy", accuracyScore)

Precision 0.6724137931034483
Accuracy 0.7229437229437229


In [ ]:
scaled_X = min_max_scaled_dataset[:,0:8]
scaled_y = min_max_scaled_dataset[:,8]

precisionScore,accuracyScore = ann_model_without_k_fold(scaled_X, scaled_y,100,20)

In [22]:
print("Precision", precisionScore)
print("Accuracy", accuracyScore)

Precision 0.6756756756756757
Accuracy 0.7445887445887446


In [ ]:
# Implementing with K Fold validation
for train_index, test_index in folds.split(scaled_X,scaled_y):
   X_train, X_test = scaled_X[train_index], scaled_X[test_index]
   y_train, y_test = scaled_y[train_index], scaled_y[test_index]
   ann_model(X_train,y_train,X_test,y_test,100,20)

In [24]:
precisionScores 

[0.6538461538461539,
 0.7777777777777778,
 0.6818181818181818,
 0.6,
 0.6190476190476191,
 0.7272727272727273,
 0.75,
 0.8095238095238095,
 0.7,
 0.8095238095238095]

In [25]:
accuracyScores

[0.7532467532467533,
 0.7792207792207793,
 0.7532467532467533,
 0.7142857142857143,
 0.7142857142857143,
 0.7792207792207793,
 0.8051948051948052,
 0.8181818181818182,
 0.7631578947368421,
 0.8289473684210527]

In [27]:
print("Avg Precision :  %.2f%% (+/- %.2f%%)" % (np.mean(precisionScores), np.std(precisionScores)))
print("Avg Accuracy : %.2f%% (+/- %.2f%%)" % (np.mean(accuracyScores), np.std(accuracyScores)))

Avg Precision :  0.71% (+/- 0.07%)
Avg Accuracy : 0.77% (+/- 0.04%)
